## Exercício 1
![alt text](ex1.png)

## Exercício 2
![alt text](ex2.png)
![alt text](ex2_2.png)

In [ ]:
hbuky

## Exercício 3
![alt text](ex3.png)

Tema ambiental escolhido: **Annual change in forest aream (hectares per year)**

\
Net change in forest area measures forest expansion (either through afforestation or natural expansion) minus deforestation

In [ ]:
import pandas as pd
import numpy as np

dados = pd.read_csv('forest-area-net-change-rate.csv')

# Lista de países que você deseja filtrar
paises_selecionados = ['Uruguay', 'Iceland', 'China', 'Chile', 'Venezuela', 'Argentina', 'Brazil', 'Colombia', 'Paraguay']

# Filtrar os dados
dados_filtrados = dados[dados['Entity'].isin(paises_selecionados)]

# Exibir os dados filtrados
display(dados_filtrados) 

,Entity,Code,Year,15.2.1 - Annual forest area change rate (%) - AG_LND_FRSTCHG
0,Afghanistan,AFG,2010,0.00
1,Afghanistan,AFG,2020,0.00
2,Africa (UN),NaN,2010,-0.49
3,Africa (UN),NaN,2020,-0.60
4,Algeria,DZA,2010,1.96
...,...,...,...,...
481,Yemen,YEM,2020,0.00
482,Zambia,ZMB,2010,-0.08
483,Zambia,ZMB,2020,-0.41
484,Zimbabwe,ZWE,2010,-0.25


,Entity,Code,Year,15.2.1 - Annual forest area change rate (%) - AG_LND_FRSTCHG
16,Argentina,ARG,2010,-0.99
17,Argentina,ARG,2020,-0.56
60,Brazil,BRA,2010,-0.74
61,Brazil,BRA,2020,-0.30
88,Chile,CHL,2010,0.56
89,Chile,CHL,2020,0.86
90,China,CHN,2010,1.26
91,China,CHN,2020,0.93
92,Colombia,COL,2010,-0.31
93,Colombia,COL,2020,-0.28
